Load API Keys

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

Load Contents from HLB Fixed Deposite Pages

In [2]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader([
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit.html?icp=hlb-en-all-footer-txt-fd",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/fixed-deposit-account.html", 
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/e-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/senior-savers-flexi-fd.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/junior-fixed-deposit.html",
  "https://www.hlb.com.my/en/personal-banking/fixed-deposit/fixed-deposit-account/foreign-fixed-deposit-account.html",
  "https://www.hlb.com.my/en/personal-banking/help-support/fees-and-charges/deposits.html",
])
data = loader.load()
data

[Document(page_content="\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n      

Split the Loaded Data into Chunks

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(data)
texts

[Document(page_content='Fixed Deposit - HLB\n\n\n\n\n\n\n\n\n\n\nFixed Deposit - HLB\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPERSONAL\n\n\nBUSINESS\n\n\n\n                            Priority\n\n\nPriority\nIslamic\nGlobal Markets\nBlue Header\nInterim Homepage\nPriority Banking Form\ntest\ntest\nLoan Cal\nLog Viewer\nasdsadsad\nstand alone page\nMaster Form Page\nMaster Form Page\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nsingapore\n\n\n\nhong kong\n\n\n\nvietnam\n\n\n\ncambodia\n\n\n\n\n\n                            ENG\n\n\nLAN A\nLAN B\nLAN C\n\n\n\n\n                            Online Banking\n                        \n\nPersonal\nBusiness\n\n\n\n\n\n\n\n\n\n\n\n\n\n                                About Us\n                            \n\nOur Story\nNews & Updates\nInvestor Relations\nCSR\nAwards\nManagement & BODs\nCareer\n\n\n\n\n                                Discover Our Products\n                            \n\nDeposits\nCredit Card\nLoans\nInsurance\nInvestment\nRemittance\

Initialize Pinecone

In [4]:
import pinecone
import os

pinecone.init(
  api_key=os.getenv("PINECONE_API_KEY"),
  environment=os.getenv("PINECONE_ENVIRONMENT")
)
index_name = "main-index"
namespace = "hlb-fixed-deposit"
if index_name not in pinecone.list_indexes():
  pinecone.create_index(
    name=index_name,
    metric="cosine",
    dimension=1536
  )
index = pinecone.Index(index_name)
index.delete(delete_all=True, namespace=namespace)

d:\Code\langchain-test\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


{}

Create Embeddings and Vectors to Insert to Pinecone Database

In [5]:
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
docs = [t.page_content for t in texts]
vector = Pinecone.from_texts(docs, embeddings, index_name=index_name, namespace=namespace)

Create Retriever for Vector Search

In [6]:
retriever = vector.as_retriever(search_type="similarity_score_threshold" ,search_kwargs={'score_threshold': 0.6, 'k': 4})
retriever

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x000001CC495E96D0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.6, 'k': 4})

Creating Prompt Template

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model="gpt-4-1106-preview", temperature=0)
prompt_template = """
  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  {chat_history}

  Reply to the following input: {input}
  By only referring the follwing document: {document}
  If you don't have enought information from the document, don't answer with your own knowledge.
  You can refer to chat history also.
  Your answer should be detailed but if you have to do calculation, don't show your work or equations, double check your answer, just provide the final value, and you don't need to show reference.
  If the bank's name is not provided in the input, answer with the assumption that the question is about HLB.
  If the bank's name is provided and it's not HLB, don't answer that question.
  You don't need to answer if the input requires you to do something creative such as writing a song or making a joke.
"""
prompt = PromptTemplate(template=prompt_template, input_variables=['input', 'document', 'chat_history'])

Storing History and Creating Chain

In [8]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from langchain.callbacks import get_openai_callback

memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="input", k=4)
chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

def ask_question(question):
  with get_openai_callback() as cb:
      document = retriever.get_relevant_documents(question)
      result = chain.run(input=question, document=document)
      print(f'{cb}\n')
      print('Retrieved Documents:')
      for i in range(len(document)):
        print(f'{i}. {document[i]}')
      print(f'\nAnswer: {result}')


Running the Chain

In [9]:
ask_question("Do you know how to calculate compound interest? is it in the document?")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  

  Reply to the following input: Do you know how to calculate compound interest? is it in the document?
  By only referring the follwing document: [Document(page_content='Terms and Conditions apply.\n\n\n\n\nMember of PIDM. Protected by PIDM up to RM250,000 for each depositor.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTenure\nInterest Rates (p.a.)\n1 month\n1.85%\n2 months\n2.25%\n3 months\n2.25%\n4 months\n2.30%\n5 months\n2.30%\n6 months\n2.30%\n7 months\n2.35%\n8 months\n2.35%\n9 months\n2.35%\n10 months\n2.35%\n11 months\n2.35%\n12 months\n2.35%\n13 - 60 months\n\n2.35%\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nImportant notice\nPlease click\xa0HERE\xa0for the exhaustive list of fees and charges.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nEnglish\n\n\n\n\n\n\n\n\n\n\n\ne-Fixed Deposit Terms and Conditions of Ac

In [ ]:
ask_question("What is fixed deposit?")

In [ ]:
ask_question("How many types of fixed deposit are there?")

In [ ]:
ask_question("tell me about the 4th one")

In [ ]:
ask_question("What are the pros and cons of fixed deposit?")

In [ ]:
ask_question("Who is jay z?")

In [ ]:
ask_question("Are you sure you dont know Jay Z? He is a very popular hip hop artist?")

In [42]:
ask_question("How many types of fixed deposit Citibank provide?")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  Human: Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.
AI: Based on the provided documents, the interest rate for a Fixed Deposit (FD) for a tenure of 7-11 months is 2.35% per annum. For an investment of RM 5000 for 9 months, the total withdrawal amount at the end of the term would be RM 5,088.13.

  Reply to the following input: How many types of fixed deposit Citibank provide?
  By only referring the follwing document: [Document(page_content="Junior Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nAn attractive interest FD account for children below the age of 18\n\n\n\n\n\n\n\n\n\n\nForeign Currency Fixed Deposit\n\n\n\n\n\n\nMore Details\n\n\n\nA convenient way to invest in foreign currency\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [ ]:
ask_question("What are the interest rates for Fixed Deposit?")

In [ ]:
ask_question("Can you find out by browsing the web?")

In [44]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  Human: Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.
AI: Based on the provided documents, the interest rate for a Fixed Deposit (FD) for a tenure of 7-11 months is 2.35% per annum. For an investment of RM 5000 for 9 months, the total withdrawal amount at the end of the term would be RM 5,088.13.
Human: How many types of fixed deposit Citibank provide?
AI: I'm sorry, but I cannot provide information on Citibank's fixed deposit products as the documents provided only contain information related to HongLeong Bank's fixed deposit accounts.
Human: Do you know how to calculate compound interest? is it in the document?
AI: The documents provided do not contain information on how to calculate compound interest.

  Reply to the follow

In [41]:
ask_question("Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  

  Reply to the following input: Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.
  By only referring the follwing document: [Document(page_content='Interest Payout\n\nReceive interest at maturity.\n\xa0\n\n\n\r\n                            More details\n\r\n                            Less details\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nFlexibility of early fixed deposit partial withdrawal without losing interest on remaining balance\n\nPartial withdrawal is in multiples of RM1,000. Outstanding balances will be\xa0shown on the monthly e-statement.\nEffective 1 January 2019, no interest shall be payable on partially withdrawn amounts and premature withdrawals of FD.\n\xa0\n\n\n\r\n                            More details\n\r\n        

In [ ]:
ask_question("Can a non-resident of Malaysia invest in CIMB SGD fixed deposit?")

In [ ]:
ask_question("Who is the CEO of CIMB bank?")

In [ ]:
ask_question("What is CIMB net profit for the last financial year?")

In [ ]:
ask_question("What if Jay Z from United States wants to invest in SGD fixed deposit. Is that ok ?")

In [ ]:
ask_question("What if Michael Jackson wants to invest in SGD Fixed Deposit. Is that ok?")

In [45]:
ask_question("Write a poem for HLB fixed deposit.")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  Human: Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.
AI: Based on the provided documents, the interest rate for a Fixed Deposit (FD) for a tenure of 7-11 months is 2.35% per annum. For an investment of RM 5000 for 9 months, the total withdrawal amount at the end of the term would be RM 5,088.13.
Human: How many types of fixed deposit Citibank provide?
AI: I'm sorry, but I cannot provide information on Citibank's fixed deposit products as the documents provided only contain information related to HongLeong Bank's fixed deposit accounts.
Human: Do you know how to calculate compound interest? is it in the document?
AI: The documents provided do not contain information on how to calculate compound interest.
Human: Lets say I want

In [ ]:
ask_question("Write a poem for HLB fixed deposit.")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  Human: Lets say I want to invest RM 5000 in Fixed Deposit for 9 months. Please calculate total withdrawal amount at the end of the term.
AI: Based on the provided documents, the interest rate for a Fixed Deposit (FD) for a tenure of 7-11 months is 2.35% per annum. For an investment of RM 5000 for 9 months, the total withdrawal amount at the end of the term would be RM 5,088.13.
Human: How many types of fixed deposit Citibank provide?
AI: I'm sorry, but I cannot provide information on Citibank's fixed deposit products as the documents provided only contain information related to HongLeong Bank's fixed deposit accounts.
Human: Do you know how to calculate compound interest? is it in the document?
AI: The documents provided do not contain information on how to calculate compound interest.
Human: Lets say I want

In [47]:
ask_question("What is the difference between fixed deposit and e-fixed deposit?")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  Human: Do you know how to calculate compound interest? is it in the document?
AI: The documents provided do not contain information on how to calculate compound interest.
Human: Lets say I want to invest RM 5000 in Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?
AI: Based on the information provided in the documents, there is no specific mention of the compound interest rate for a Fixed Deposit of RM 5000 for 36 months. Therefore, I cannot calculate the total withdrawal amount at the end of the term with the given interest rate of 5% compound interest.
Human: Write a poem for HLB fixed deposit.
AI: Your request for a poem about HLB fixed deposit cannot be fulfilled as it requires a creat

In [48]:
ask_question("Write a short story about HLB fixed deposit.")



> Entering new LLMChain chain...
Prompt after formatting:

  You are a helpful assistant who should reply to inputs related only to the HongLeong Bank aka HLB.

  Chat History:
  Human: Lets say I want to invest RM 5000 in Fixed Deposit for 36 months and the interest rate is 5%. Please calculate total withdrawal amount at the end of the term. Take note the interest rate is compund interest?
AI: Based on the information provided in the documents, there is no specific mention of the compound interest rate for a Fixed Deposit of RM 5000 for 36 months. Therefore, I cannot calculate the total withdrawal amount at the end of the term with the given interest rate of 5% compound interest.
Human: Write a poem for HLB fixed deposit.
AI: Your request for a poem about HLB fixed deposit cannot be fulfilled as it requires a creative task which is outside the scope of the provided documents and instructions.
Human: how many types of saving accounts are there in a general bank?
AI: Based on the docu